In [1]:
import cPickle as pickle
import string
import os

%matplotlib inline
import numpy as np
import random
import time

import logging
from logging import info

from multiprocessing import Pool as ThreadPool
import itertools

In [2]:
root = logging.getLogger()
for handler in root.handlers[:]:
    root.removeHandler(handler)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO) # adds a default StreamHanlder
#root.addHandler(logging.StreamHandler())

In [5]:
root_location = "/home/shalaby/data/"
preprocessed_data_location = root_location + "preprocessed_data/exstended_pv_abs_desc_claims_full_chunks/"
new_preprocessed_data_location = root_location + "preprocessed_data/extended_pv_abs_desc_claims_full_chunks_merged/"

if not os.path.exists(new_preprocessed_data_location):
    os.makedirs(new_preprocessed_data_location)

In [6]:
LINES_PER_DOCUMENT = 34
BATCH_DOCS_SIZE = 10000
NEW_BATCH_DOCS_SIZE = 50000
BATCH_SIZE_LINES = LINES_PER_DOCUMENT * BATCH_DOCS_SIZE
NEW_BATCH_SIZE_LINES = LINES_PER_DOCUMENT * NEW_BATCH_DOCS_SIZE


TRAINING_PREPROCESSED_FILES_PREFIX = preprocessed_data_location + "extended_pv_training_docs_data_preprocessed-"
VALIDATION_PREPROCESSED_FILES_PREFIX = preprocessed_data_location + "validation_docs_data_preprocessed-"
TEST_PREPROCESSED_FILES_PREFIX = preprocessed_data_location + "test_docs_data_preprocessed-"


TRAINING_NEW_PREPROCESSED_FILES_PREFIX = new_preprocessed_data_location + "extended_pv_training_docs_data_preprocessed-"
VALIDATION_NEW_PREPROCESSED_FILES_PREFIX = new_preprocessed_data_location + "validation_docs_data_preprocessed-"
TEST_NEW_PREPROCESSED_FILES_PREFIX = new_preprocessed_data_location + "test_docs_data_preprocessed-"


In [7]:
FILE_PREFIX_TO_READ = TRAINING_PREPROCESSED_FILES_PREFIX
FILE_PREFIX_TO_WRITE = TRAINING_NEW_PREPROCESSED_FILES_PREFIX

In [8]:
FILE_PREFIX_TO_READ

'/home/shalaby/data/preprocessed_data/extended_pv_abs_desc_claims_full_chunks/extended_pv_training_docs_data_preprocessed-'

In [9]:
curr_index = 0
curr_read_file_index = 0
curr_write_file_index = 0

read_file = None
write_file = None

def read_line():
    global read_file, curr_read_file_index
    while True:
        if read_file is None:
            info("Reading new file for batch {}".format(curr_read_file_index))
            if os.path.exists(FILE_PREFIX_TO_READ + str(curr_read_file_index)):
                read_file = open(FILE_PREFIX_TO_READ + str(curr_read_file_index), "r")
            else: 
                raise StopIteration()
        for line in read_file:
            yield line
            
        curr_read_file_index += BATCH_DOCS_SIZE
        read_file.close()
        read_file = None

def write_line(line):
    global write_file, curr_write_file_index, curr_index
    if write_file is None:
        info("Writing to a new file for batch {}".format(curr_write_file_index))
        write_file = open(FILE_PREFIX_TO_WRITE + str(curr_write_file_index), "w")
    write_file.write(line)
    curr_index += 1
    if curr_index % NEW_BATCH_SIZE_LINES == 0:
        curr_write_file_index += NEW_BATCH_DOCS_SIZE
        write_file.close()
        write_file = None
    

for line in read_line():
    write_line(line)

2017-04-05 01:25:04,402 : INFO : Reading new file for batch 0
2017-04-05 01:25:04,569 : INFO : Writing to a new file for batch 0
2017-04-05 01:26:06,713 : INFO : Reading new file for batch 10000


KeyboardInterrupt: 